# pandas数据处理

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

## 1、删除重复元素

使用duplicated()函数检测重复的行，返回元素为布尔类型的Series对象，每个元素对应一行，如果该行不是第一次出现，则元素为True

In [4]:
def make_df(index, cols):
    df = DataFrame({col: [col + str(i) for i in index] for col in cols})
    df.index=index
    return df

In [5]:
df1 = make_df(list('1234'), list('ABCD'))

In [6]:
df1.iloc[0] = df1.iloc[1]

In [7]:
df1

,A,B,C,D
1,A2,B2,C2,D2
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


In [8]:
# pandas提供了检测重复数据的方法duplicated
df1.duplicated()

1    False
2     True
3    False
4    False
dtype: bool

In [17]:
df1.loc['2','D'] = 'D1'
df1

,A,B,C,D
1,A2,B2,C2,D2
2,A2,B2,C2,D1
3,A3,B3,C3,D3
4,A4,B4,C4,D4


In [10]:
df1.duplicated()

1    False
2    False
3    False
4    False
dtype: bool

In [11]:
# subset子集
df1.duplicated(subset=['A', 'B', 'C'])

1    False
2     True
3    False
4    False
dtype: bool

In [24]:
# 原生的过滤方法, 把df.duplicated() 的结果当成条件去过滤
df1.loc[df1.duplicated()]

,A,B,C,D
2,A2,B2,C2,D2


In [22]:
df1.loc['2','D'] = 'D2'
df1

,A,B,C,D
1,A2,B2,C2,D2
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


In [23]:
# 逻辑运算,
# numpy 提供了逻辑运算
np.logical_not(df1.duplicated())

1     True
2    False
3     True
4     True
dtype: bool

In [16]:
df1.loc[np.logical_not(df1.duplicated())]

,A,B,C,D
1,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


In [17]:
df1.loc[~(df1.duplicated())]

,A,B,C,D
1,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


使用drop_duplicates()函数删除重复的行

In [27]:
df1.drop_duplicates(keep='last', subset=['A','B','C'])

,A,B,C,D
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


## 2. 映射

映射的含义：创建一个映射关系列表，把values元素和一个特定的标签或者字符串绑定

需要使用字典：

`map = {
    'label1':'value1',
    'label2':'value2',
    ...
    }
`

包含三种操作：

- replace()函数：替换元素
- 最重要：map()函数：新建一列
- rename()函数：替换索引

### 1) replace()函数：替换元素

使用replace()函数，对values进行替换操作

In [28]:
index = ['张三', '李四', '王五', '赵六']
columns = ['语文', '数学', '英语', 'python']
data = np.random.randint(0, 150, size=(4, 4))
df2 = DataFrame(data=data, index=index, columns=columns)
df2

,语文,数学,英语,python
张三,69,91,80,99
李四,2,132,91,147
王五,69,40,19,135
赵六,63,136,108,2


首先定义一个字典

In [29]:
mapping = {2: 120, 19: 119}

In [30]:
df2.replace(mapping)

,语文,数学,英语,python
张三,69,91,80,99
李四,120,132,91,147
王五,69,40,119,135
赵六,63,136,108,120


调用.replace()

replace还经常用来替换NaN元素

In [32]:
mapping = {2: np.nan, 19: np.nan}
df2.replace(mapping, inplace=True)
df2

,语文,数学,英语,python
张三,69.0,91.0,80.0,99.0
李四,NaN,132.0,91.0,147.0
王五,69.0,40.0,NaN,135.0
赵六,63.0,136.0,108.0,NaN


In [40]:
df2.replace({np.nan: 0})

,语文,数学,英语,python
张三,69.0,91.0,80.0,99.0
李四,0.0,132.0,91.0,147.0
王五,69.0,40.0,0.0,135.0
赵六,63.0,136.0,108.0,0.0


============================================

练习19：

    假设张三李四的课表里有满分的情况，老师认为是作弊，把所有满分的情况（包括150,300分）都记0分，如何实现？

============================================

In [43]:
df2.loc['张三','英语'] = 150
df2.loc['李四', 'python'] = 300

In [45]:
df2

,语文,数学,英语,python
张三,69.0,91.0,150.0,99.0
李四,NaN,132.0,91.0,300.0
王五,69.0,40.0,NaN,135.0
赵六,63.0,136.0,108.0,NaN


In [46]:
mapping = {150:0, 300:0}
df2.replace(mapping)

,语文,数学,英语,python
张三,69.0,91.0,0.0,99.0
李四,NaN,132.0,91.0,0.0
王五,69.0,40.0,NaN,135.0
赵六,63.0,136.0,108.0,NaN


### 2) map()函数：新建一列

使用map()函数，由已有的列生成一个新列

适合处理某一单独的列。

In [47]:
df2

,语文,数学,英语,python
张三,69.0,91.0,150.0,99.0
李四,NaN,132.0,91.0,300.0
王五,69.0,40.0,NaN,135.0
赵六,63.0,136.0,108.0,NaN


In [49]:
df2.loc[:, '计算机'] = np.array
df2

,语文,数学,英语,python,计算机
张三,69.0,91.0,150.0,99.0,<built-in function array>
李四,NaN,132.0,91.0,300.0,<built-in function array>
王五,69.0,40.0,NaN,135.0,<built-in function array>
赵六,63.0,136.0,108.0,NaN,<built-in function array>


In [57]:
# 根据已有列生成函数
df2['计算机'] = df2['python'].map({99:100,300:190,135:10, np.nan:12})
df2

,语文,数学,英语,python,计算机
张三,69.0,91.0,150.0,99.0,100
李四,NaN,132.0,91.0,300.0,190
王五,69.0,40.0,NaN,135.0,10
赵六,63.0,136.0,108.0,NaN,12


仍然是新建一个字典

map()函数中可以使用lambda函数

In [60]:
df2['python'] = df2['python'].map(lambda score: score-50)
df2

,语文,数学,英语,python,计算机
张三,69.0,91.0,150.0,-1.0,100
李四,NaN,132.0,91.0,200.0,190
王五,69.0,40.0,NaN,35.0,10
赵六,63.0,136.0,108.0,NaN,12


In [62]:
# 创建自定义函数
def covert(score):
    if 300>score>150:
        return score - 50
    elif 150>score>100:
        return score - 30
    else:
        return score +50

In [98]:
df2['python'] = df2['python'].map(covert)

df2

,语文,数学,英语,python,计算机
张三,69.0,91.0,150.0,99.0,100
李四,NaN,132.0,91.0,200.0,190
王五,69.0,40.0,NaN,135.0,10
赵六,63.0,136.0,108.0,NaN,12


transform()和map()类似

In [99]:
df2['python'].transform(covert)

张三    149.0
李四    150.0
王五    105.0
赵六      NaN
Name: python, dtype: float64

使用map()函数新建一个新列

============================================

练习20：

    新增两列，分别为张三、李四的成绩状态，如果分数低于90，则为"failed"，如果分数高于120，则为"excellent"，其他则为"pass"
    
    【提示】使用函数作为map的参数

============================================

### 3) rename()函数：替换索引

In [101]:
df2

,语文,数学,英语,python,计算机
张三,69.0,91.0,150.0,99.0,100
李四,NaN,132.0,91.0,200.0,190
王五,69.0,40.0,NaN,135.0,10
赵六,63.0,136.0,108.0,NaN,12


In [105]:
# 用来替换索引的.
df2.rename(columns={'语文':'Chinese','英语':'English'}, index={'张三': 'Zhang San'})

,Chinese,数学,English,python,计算机
Zhang San,69.0,91.0,150.0,99.0,100
李四,NaN,132.0,91.0,200.0,190
王五,69.0,40.0,NaN,135.0,10
赵六,63.0,136.0,108.0,NaN,12


仍然是新建一个字典

使用rename()函数替换行索引

## 3. 异常值检测和过滤

使用describe()函数查看每一列的描述性统计量

In [107]:
#
df2.describe()

,语文,数学,英语,python,计算机
count,3.000000,4.000000,3.000000,3.000000,4.000000
mean,67.000000,99.750000,116.333333,144.666667,78.000000
std,3.464102,44.724155,30.369941,51.189192,85.650452
min,63.000000,40.000000,91.000000,99.000000,10.000000
25%,66.000000,78.250000,99.500000,117.000000,11.500000
50%,69.000000,111.500000,108.000000,135.000000,56.000000
75%,69.000000,133.000000,129.000000,167.500000,122.500000
max,69.000000,136.000000,150.000000,200.000000,190.000000


使用std()函数可以求得DataFrame对象每一列的标准差

根据每一列的标准差，对DataFrame元素进行过滤。

借助any()函数, 测试是否有True，有一个或以上返回True，反之返回False

对每一列应用筛选条件,去除标准差太大的数据

删除特定索引df.drop(labels,inplace = True)

============================================

练习21：

    新建一个形状为10000*3的标准正态分布的DataFrame(np.random.randn)，去除掉所有满足以下情况的行：其中任一元素绝对值大于3倍标准差

============================================

In [110]:
df =DataFrame(data=np.random.randn(10000,3))
df

,0,1,2
0,-1.515160,1.097663,-0.116185
1,0.692254,-0.130619,0.078411
2,1.601445,-0.825246,-0.116317
3,1.400039,0.085612,-0.527065
4,0.348175,-0.256927,0.933440
...,...,...,...
9995,1.175575,1.270885,-1.289761
9996,0.109138,-0.734859,1.260807
9997,0.584011,0.218021,-0.155717
9998,-0.427844,-0.647963,0.170441


In [111]:
df.std()

0    1.009328
1    1.000643
2    1.010147
dtype: float64

In [ ]:
# 把绝对值大于三倍标准偏差的行找出来
# dataframe和series做比较运算, 规则: series的索引和dataframe的列索引 

## 4. 排序

使用.take()函数排序

可以借助np.random.permutation()函数随机排序

### 随机抽样

In [18]:
index = ['张三', '李四', '王五', '赵六']
columns = ['语文', '数学', '英语', 'Python']
data = np.random.randint(0, 150, size=(4, 4))
df = DataFrame(data=data, index=index, columns=columns)

,语文,数学,英语,Python
张三,35,55,136,55
李四,56,44,12,47
王五,42,92,42,111
赵六,72,74,38,75


In [ ]:
# 抽样有两种: 有放回抽样和无放回抽样
# 有放回抽样,样本可能重复出现
# 无放回抽样, 样本不可能重复出现

In [19]:
df

,语文,数学,英语,Python
张三,35,55,136,55
李四,56,44,12,47
王五,42,92,42,111
赵六,72,74,38,75


In [22]:
df.take([2, 0, 1, 3], axis=0) # 默认是0

,语文,数学,英语,Python
王五,42,92,42,111
张三,35,55,136,55
李四,56,44,12,47
赵六,72,74,38,75


In [26]:
# 有放回抽样
df.take(np.random.randint(0, 4, size=4))

,语文,数学,英语,Python
张三,35,55,136,55
赵六,72,74,38,75
赵六,72,74,38,75
赵六,72,74,38,75


In [29]:
# 无放回抽样, take中的索引不能出现重复值
# 借助, np.random.premutation()
np.random.permutation([0, 1, 2, 3])

array([1, 3, 0, 2])

In [32]:
df.take(np.random.permutation(np.arange(4)))

,语文,数学,英语,Python
张三,35,55,136,55
赵六,72,74,38,75
李四,56,44,12,47
王五,42,92,42,111


In [34]:
# 对numpy中ndarray抽样,怎么抽
n = np.arange(100)
n

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [54]:
np.random.permutation(n)[:10]

array([20, 55, 56,  1, 30, 12, 80, 18, 86, 94])

In [35]:
# 打乱数据后取前十个
np.random.shuffle(n)
n[:10]

array([ 3, 59,  2, 90, 82, 54, 39, 76,  7, 84])

当DataFrame规模足够大时，直接使用np.random.randint()函数，就配合take()函数实现随机抽样

============================================
练习22：

   假设有张三李四王老五的期中考试成绩ddd2，对着三名同学随机排序

============================================

## 5. 数据聚合【重点】

数据聚合是数据处理的最后一步，通常是要使每一个数组生成一个单一的数值。

数据分类处理：

 - 分组：先把数据分为几组
 - 用函数处理：为不同组的数据应用不同的函数以转换数据
 - 合并：把不同组得到的结果合并起来
 
数据分类处理的核心：
     groupby()函数

如果想使用color列索引，计算price1的均值，可以先获取到price1列，然后再调用groupby函数，用参数指定color这一列

In [2]:
df = DataFrame({'color':['red','white','red','cyan','cyan','green','white','cyan'],
                'price':np.random.randint(0,8,size = 8),
                'weight':np.random.randint(50,55,size = 8)})
df

,color,price,weight
0,red,0,51
1,white,3,51
2,red,3,52
3,cyan,6,51
4,cyan,3,54
5,green,7,53
6,white,7,52
7,cyan,1,50


In [5]:
# 根b据颜色进行分组
df.groupby(by=['color','price']).groups

{('cyan', 1): [7], ('cyan', 3): [4], ('cyan', 6): [3], ('green', 7): [5], ('red', 0): [0], ('red', 3): [2], ('white', 3): [1], ('white', 7): [6]}

In [4]:
# 通过groups查看分组情况
df.groupby(by=['color']).groups

{'cyan': [3, 4, 7], 'green': [5], 'red': [0, 2], 'white': [1, 6]}

In [64]:
# 查看各组的价格平均值
# 先取数据,在聚合 , 推荐写法
price_mean = df.groupby(by='color')[['price']].mean()
price_mean

,price
color,
cyan,4.0
green,5.0
red,3.5
white,2.0


In [63]:
# 先聚合再取数据
df.groupby(by='color').mean()[['price']]

,price
color,
cyan,4.0
green,5.0
red,3.5
white,2.0


In [65]:
display(df, price_mean)

,color,price,weight
0,red,4,51
1,white,0,52
2,red,3,51
3,cyan,7,54
4,cyan,0,50
5,green,5,51
6,white,4,53
7,cyan,5,51


,price
color,
cyan,4.0
green,5.0
red,3.5
white,2.0


In [66]:
df.merge(price_mean, left_on='color', right_index=True, suffixes=['', '_mean'])

,color,price,weight,price_mean
0,red,4,51,3.5
2,red,3,51,3.5
1,white,0,52,2.0
6,white,4,53,2.0
3,cyan,7,54,4.0
4,cyan,0,50,4.0
7,cyan,5,51,4.0
5,green,5,51,5.0


使用.groups属性查看各行的分组情况：

============================================

练习23：

   假设菜市场张大妈在卖菜，有以下属性：
   
   菜品(item)：萝卜，白菜，辣椒，冬瓜
   
   颜色(color)：白，青，红
   
   重量(weight)
   
   价格(price)
   
1. 要求以属性作为列索引，新建一个ddd
2. 对ddd进行聚合操作，求出颜色为白色的价格总和
3. 对ddd进行聚合操作，求出萝卜的所有重量(包括白萝卜，胡萝卜，青萝卜）以及平均价格
4. 使用merge合并总重量及平均价格

============================================

In [103]:
# color_df = DataFrame(data=['白','青','红'], columns=['color'], index=list('012'))
# color_df

In [102]:
ddd = DataFrame({'item':['萝卜','白菜','辣椒','冬瓜']*3,
                 'color': ['白','青','红']*4,
                'price':np.random.randint(0,8,size = 12),
                'weight':np.random.randint(50,55,size = 12)})
ddd

,item,color,price,weight
0,萝卜,白,4,50
1,白菜,青,4,50
2,辣椒,红,3,53
3,冬瓜,白,7,53
4,萝卜,青,0,53
5,白菜,红,0,54
6,辣椒,白,2,51
7,冬瓜,青,7,54
8,萝卜,红,0,50
9,白菜,白,0,50


In [115]:
# 2. 对ddd进行聚合操作，求出颜色为白色的价格总和
ddd.groupby(by='color')[['price']].mean()

,price
color,
白,3.25
红,2.25
青,3.50


In [117]:
# 3. 对ddd进行聚合操作，求出萝卜的所有重量(包括白萝卜，胡萝卜，青萝卜）以及平均价格
weight = ddd.groupby(by='item')[['weight']].sum()
mean_luobo = ddd.groupby(by='item')[['price']].mean()
display(weight, mean_luobo)

,weight
item,
冬瓜,160
白菜,154
萝卜,153
辣椒,154


,price
item,
冬瓜,6.666667
白菜,1.333333
萝卜,1.333333
辣椒,2.666667


In [121]:
# 4. 使用merge合并总重量及平均价格
ddd = ddd.merge(weight, left_on='item', right_index=True, suffixes=['', '_sum'])

In [122]:
ddd.merge(mean_luobo, left_on='item', right_index=True, suffixes=['', '_mean'])

,item,color,price,weight,weight_sum,price_mean
0,萝卜,白,4,50,153,1.333333
4,萝卜,青,0,53,153,1.333333
8,萝卜,红,0,50,153,1.333333
1,白菜,青,4,50,154,1.333333
5,白菜,红,0,54,154,1.333333
9,白菜,白,0,50,154,1.333333
2,辣椒,红,3,53,154,2.666667
6,辣椒,白,2,51,154,2.666667
10,辣椒,青,3,50,154,2.666667
3,冬瓜,白,7,53,160,6.666667


## 6.0 高级数据聚合

可以使用pd.merge()函数将聚合操作的计算结果添加到df的每一行  
使用groupby分组后调用加和等函数进行运算，让后最后可以调用add_prefix()，来修改列名

### 可以使用transform和apply实现相同功能

在transform或者apply中传入函数即可

In [123]:
ddd.groupby(by='item')[['weight']].transform(sum)

,weight
0,153
4,153
8,153
1,154
5,154
9,154
2,154
6,154
10,154
3,160


In [126]:
ddd.groupby(by='item')[['weight']].apply(sum)

,weight
item,
冬瓜,160
白菜,154
萝卜,153
辣椒,154
